In [1]:
from roboflow import Roboflow

In [2]:
rf = Roboflow(api_key="nz2w1UFnyFrM7e73WOzh")
project = rf.workspace("nyu-figsb").project("basketballdetection-cki6r")
project.version(21).deploy("yolov8", ".")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.17, to fix: `pip install ultralytics==8.0.196`
View the status of your deployment at: https://app.roboflow.com/nyu-figsb/basketballdetection-cki6r/21
Share your model with the world at: https://universe.roboflow.com/nyu-figsb/basketballdetection-cki6r/model/21


In [1]:
from model import MODEL
import time
from PIL import Image
import torch
import cv2
from tqdm import tqdm
from video_utils import initialize_video_capture
from ultralytics import YOLO
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
cap,fps, width, height, total_frames = initialize_video_capture("full_videos/NM9_fvYsWME.mp4")
ret, orig_img = cap.read()
batch_size = 32
total_imgs = 2 ** 10
num_batches = int(total_imgs / batch_size)

model = MODEL(model_path = "weights/detect_large.pt",
              device = device)
_, img = cap.read()

In [3]:
yolo = YOLO("weights/detect_large.pt")

In [6]:
batch_range = tqdm(range(num_batches), desc='Processing Batches')
for i in batch_range:
    start_time = time.time()
    img_list = []
    for _ in range(batch_size):
        ret, orig_img = cap.read()
        if ret:
            img_list.append(orig_img)
            
    model.predict(img_list)
    
    elapsed_time = time.time() - start_time  # Elapsed time for batch
    fps = batch_size / elapsed_time  # Calculate fps based on batches processed
    batch_range.set_postfix(fps=f"{fps:.2f} fps", refresh=True)  

Processing Batches: 100%|██████████| 32/32 [00:09<00:00,  3.44it/s, fps=112.31 fps]


In [5]:
batch_range = tqdm(range(num_batches), desc='Processing Batches')
for i in batch_range:
    start_time = time.time()
    img_list = []
    for _ in range(batch_size):
        ret, orig_img = cap.read()
        if ret:
            img_list.append(orig_img)
            
    ultralytics_output = yolo(img_list, device=1, verbose = False)
    
    elapsed_time = time.time() - start_time  # Elapsed time for batch
    fps = batch_size / elapsed_time  # Calculate fps based on batches processed
    batch_range.set_postfix(fps=f"{fps:.2f} fps", refresh=True)  

Processing Batches: 100%|██████████| 32/32 [00:20<00:00,  1.60it/s, fps=62.72 fps]
